In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os
input_file = 'mariel-1.npy'
#input_file = 'carrie-10-min-scaled.npy'
#input_file = 'carrie-10-mins.npy'

X0 = np.load(os.path.join('data','npy', input_file))

if input_file == 'mariel-1.npy':
    X0 = X0.transpose((1,0,2))
    X0 = X0[1300:7000]
    X0[:,:,2] *= -1
    X = X0.copy()
    X -= X.mean(axis=(0,1))
    X *= 0.5
elif input_file == 'carrie-10-min-scaled.npy':
    X0 = X0.transpose((1,0,2))
    X = X0.copy()
    X[:,:,1] = X0[:,:,2]
    X[:,:,2] = X0[:,:,1]
    X = X[100:]
    X[:,:,:2] -= 0.5
if input_file == 'carrie-10-mins.npy':
    X0 = X0.transpose((1,0,2))
    X0 = X0[100:]
    X0[:,:,2] *= -1
    X = X0.copy()
    X -= X.mean(axis=(0,1))
    X *= 0.25
    
    
print(X.shape)

In [ ]:
plt.plot(X.mean(axis=1));
plt.figure()
plt.hist([X[:,:,0].flatten(),X[:,:,1].flatten(),X[:,:,2].flatten()], histtype='step');

In [ ]:
print(X.max(axis=(0,1)))
print(X.min(axis=(0,1)))

In [ ]:
print(X.mean(axis=(0,1)))
print(X.std(axis=(0,1)))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from mpl_toolkits.mplot3d.art3d import juggle_axes
from matplotlib import animation
from IPython.display import HTML

# ask matplotlib to plot up to 2^128 frames in animations
matplotlib.rcParams['animation.embed_limit'] = 2**128

def update_points(time, points, df):
  '''
  Callback function called by plotting function below. Mutates the vertex
  positions of each value in `points` so the animation moves
  @param int time: the index of the time slice to visualize within `df`
  @param mpl_toolkits.mplot3d.art3d.Path3DCollection points: the actual
    geometry collection whose internal values this function mutates to move
    the displayed points
  @param numpy.ndarray df: a numpy array with the following three axes:
    df.shape[0] = n_vertices
    df.shape[1] = n_time_slices
    df.shape[2] = n_dimensions
  '''
  points._offsets3d = juggle_axes(df[time,:,0], df[time,:,1], df[time,:,2], 'z')

def animate(seq, frames=None, axis_min=-.5, axis_max=0.5, speed=45):
    if frames is None:
        frames = len(seq)
    fig = plt.figure()
    ax = p3.Axes3D(fig)
    ax.set_xlim(axis_min, axis_max)
    ax.set_ylim(axis_min, axis_max)
    ax.set_zlim(axis_min, axis_max*1.5)
    points = ax.scatter(seq[0,:,0], seq[0,:,1], seq[0,:,2], depthshade=False)
    return animation.FuncAnimation(
        fig,
        update_points,
        frames,
        interval=speed,
        fargs=(points, seq),
        blit=False,
    ).to_jshtml()

def plot_pose(x):
    fig = plt.figure()
    ax = p3.Axes3D(fig)
    ax.set_xlim(-0.5,0.5)
    ax.set_ylim(-0.5,0.5)
    ax.set_zlim(-0.5,0.5*1.5)
    ax.scatter(x[:,0], x[:,1], x[:,2])

In [ ]:
HTML(animate(X[100:300]))

In [ ]:
import os
import tensorflow as tf
# config for Gatsby cores
if 'gatsby' in os.environ['HOSTNAME']:

    # specify target gpu device
    os.environ['CUDA_VISIBLE_DEVICES'] = '0' # either '0' or '1' to utilize Titan X GPUs

    # allow dynamic GPU allocation
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    
import keras.backend as K
from keras import layers
from keras.models import Model

In [ ]:
def mk_model(lookback=60, npred=1, n_cells=(32,32,32)):
    K.clear_session()
    
    input_layer = layers.Input((lookback, X.shape[1], X.shape[2]))
    H = input_layer
    
    H = layers.Reshape((lookback, X.shape[1]*X.shape[2]))(H)
    
    for nc in n_cells[:-1]:
        H = layers.CuDNNLSTM(nc, return_sequences=True)(H)
    H = layers.CuDNNLSTM(n_cells[-1])(H)
    H = layers.Dense(npred*X.shape[1]*X.shape[2], activation='tanh')(H)
    H = layers.Reshape((npred,X.shape[1],X.shape[2]))(H)
    output_layer = H
    
    model = Model(input_layer, output_layer)
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
lookback = 32
npred = 4
model = mk_model(lookback, npred, n_cells=(64,64,64))
model.summary()

losses = []

In [ ]:
K.set_value(model.optimizer.lr, 1e-4)

In [ ]:
# bs=128, lr=1e-4, no rotation should get interesting results in about 600 epochs
epochs = 160
batch_size = 128
do_rotation = False

nbatches = (len(X)-batch_size*lookback-npred)//batch_size

idxs = np.arange(len(X))
#nbatches = (len(X)-1)//(batch_size*lookback)
print("Nbatches =", nbatches)
for iepoch in range(epochs):
    avg_loss = 0
    offsets = np.random.choice(len(X)-batch_size*lookback-npred, replace=False, size=(nbatches,batch_size))
    for ibatch in range(nbatches):
        batch_idxs = offsets[ibatch].repeat(lookback).reshape(batch_size,lookback) + np.arange(lookback)
        truth_idxs = offsets[ibatch].repeat(npred).reshape(batch_size,npred) + np.arange(npred) + lookback
        
        if do_rotation:
            theta = 2*np.pi*np.random.rand()
            c,s = np.cos(theta),np.sin(theta)
            rot = np.array([[c,-s,0],[s,c,0],[1,0,0]])

            x_train = rot.dot(X[batch_idxs].transpose((0,1,3,2))).transpose((1,2,3,0))
            y_train = rot.dot(X[truth_idxs].transpose((0,1,3,2))).transpose((1,2,3,0))
        else:
            x_train = X[batch_idxs]
            y_train = X[truth_idxs]
            
        avg_loss += model.train_on_batch(x_train, y_train)
    losses.append(avg_loss/nbatches)
    print("Epoch %d/%d: L=%.3e"%(iepoch,epochs,losses[-1]))

In [ ]:
plt.plot(losses)
plt.yscale('log')

In [ ]:
iprompt = np.random.randint(len(X)-batch_size*lookback-1)
ngen = 4*lookback

gen_seq = np.zeros((lookback+ngen, X.shape[1], X.shape[2]))
gen_seq[:lookback] = X[iprompt:iprompt+lookback]

for i in range(ngen):
    next_frame = model.predict(np.expand_dims(gen_seq[i:i+lookback],0))[0,0]
    gen_seq[i+lookback] = next_frame

In [ ]:
HTML(animate(gen_seq))